
# Extracción de Datos a la Capa Bronze en Azure Databricks

Este notebook implementa el proceso de extracción de datos desde las tablas operativas del sistema de gestión de internamientos clínicos hacia la capa Bronze del Data Lakehouse en Azure, utilizando Delta Lake con tablas gestionadas.


Consulta a las tablas del Origen de datos y traerla a Dataframe
Definimos una funcion para conectarnos a SQL Server y cargar todo a un Dataframe Generico


In [0]:
def leer_tablas_sql_azure(lista_tablas, lista_modos_carga, fecha_corte, url_conexion, usuario, password,
                          esquemas=None, columna_incremental="fecalta"):
    """
    Lee múltiples tablas desde Azure SQL Database, usando carga full o incremental por tabla.

    Parámetros:
    - lista_tablas: lista de nombres de tablas a leer.
    - lista_modos_carga: lista del mismo tamaño que lista_tablas con 'full' o 'incremental'.
    - fecha_corte: fecha para filtro incremental.
    - url_conexion: cadena JDBC a Azure SQL.
    - usuario: nombre de usuario de conexión.
    - password: contraseña.
    - esquemas: dict opcional {tabla: esquema}, por defecto usa 'dbo'.
    - columna_incremental: nombre de la columna de fecha para carga incremental.

    Retorna:
    - None. Crea variables globales con los DataFrames.
    """

    if len(lista_tablas) != len(lista_modos_carga):
        raise ValueError("Las listas 'lista_tablas' y 'lista_modos_carga' deben tener la misma longitud.")

    properties = {
        "user": usuario,
        "password": password,
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    for i, nombre_tabla in enumerate(lista_tablas):
        modo_carga = lista_modos_carga[i].lower()
        esquema = esquemas.get(nombre_tabla, "dbo") if esquemas else "dbo"
        tabla_completa = f"{esquema}.{nombre_tabla}"
        query = ""
        base_query = f"(SELECT * FROM {tabla_completa}) AS temp"

        try:
            # Validar existencia de columna incremental
            df_sample = spark.read.jdbc(url=url_conexion, table=base_query, properties=properties).limit(10)

            if modo_carga == "incremental":
                if columna_incremental in df_sample.columns:
                    query = f"(SELECT * FROM {tabla_completa} WHERE {columna_incremental} = '{fecha_corte}') AS temp"
                else:
                    print(f"⚠️  Tabla '{tabla_completa}' sin columna '{columna_incremental}'. Se leerá completa.")
                    query = base_query
            else:
                query = base_query

            df = spark.read.jdbc(url=url_conexion, table=query, properties=properties)
            globals()[nombre_tabla] = df

            print(f"✅ DataFrame '{nombre_tabla}' ({modo_carga}) cargado desde '{tabla_completa}'.")

        except Exception as e:
            print(f"❌ Error al leer la tabla '{tabla_completa}': {e}")


In [0]:
# Parámetros de conexión
url_conexion = dbutils.secrets.get("scope-dev", "secret-sql-ventas-url")
usuario = dbutils.secrets.get("scope-dev", "secret-sql-ventas-user")
password = dbutils.secrets.get("scope-dev", "secret-sql-ventas-password")
fecha_carga = "2025-08-01"
#properties = {"user": "admin01juls", "password": "287719Julius@12", "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}

# Lista de tablas a leer
lista_tablas = ["subcategoria", "categoria", "producto", "ubigeo","segmento","mercado", "SECTORECONOMICO", "PEDIDO",
                "cliente", "VENDEDOR", "MODALIDADENVIO", "MODALIDADVENTA", "MONEDA",  "mediopago" , "PRIORIDADPEDIDO"]
esquemas = {}
modos = ["full", "full", "full", "full", "full", "full", "full", "incremental", "full","full", "full", "full", "full", "full", "full"]

# Llamar la función para crear DataFrames individuales
leer_tablas_sql_azure(lista_tablas, modos, fecha_carga, url_conexion, usuario, password, esquemas, columna_incremental="FECCARGA")

In [0]:
# BRONZE - limpieza y estandarización mínima
from typing import Dict, List, Tuple, Optional, Union
from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    col, when, lit, to_timestamp, year, month, dayofmonth
)
from pyspark.sql.types import (
    DataType, DoubleType, FloatType, IntegerType, LongType, ShortType,
    DecimalType, StringType, TimestampType
)

def _bronze_zero_for_type(dt: DataType):
    if isinstance(dt, (DoubleType, FloatType, DecimalType)):
        return lit(0.0)
    if isinstance(dt, (IntegerType, LongType, ShortType)):
        return lit(0)
    if isinstance(dt, StringType):
        return lit("")
    if isinstance(dt, TimestampType):
        return lit(None).cast("timestamp")
    return lit(0)

def limpiar_bronze(
    df: DataFrame,
    casts: Optional[Dict[str, str]] = None,
    date_col: Optional[str] = None,
    date_fmt: str = "yyyy-MM-dd HH:mm:ss",
    extract_date_parts: bool = True,
    fillna_map: Optional[Dict[str, object]] = None,
    dedup_subset: Optional[List[str]] = None,
    non_negative_cols: Optional[List[str]] = None,
    drop_cols: Optional[List[str]] = None,
    compute_metrics: bool = True,                 # NUEVO: evita counts si no los necesitas
    output: str = "both"                          # NUEVO: "df" o "both"
) -> Union[DataFrame, Tuple[DataFrame, Dict]]:
    """
    Limpieza típica de Bronze:
    - Casts de tipos.
    - Parse/normalización de fecha y particiones (year/month/day).
    - Relleno de nulos (incluye timestamps).
    - Corrección de negativos en columnas numéricas.
    - Drop de columnas innecesarias.
    - Deduplicación (subset o todas las columnas).

    Retorna:
      - si output="df": DataFrame
      - si output="both": (DataFrame, dq_metrics)
    """
    dq = {}

    # Métrica de entrada
    if compute_metrics:
        dq["rows_in"] = df.count()

    # 1) Casts
    if casts:
        for c, t in casts.items():
            if c in df.columns:
                if t.lower() == "timestamp" and not isinstance(df.schema[c].dataType, TimestampType):
                    df = df.withColumn(c, to_timestamp(col(c), date_fmt))
                else:
                    df = df.withColumn(c, col(c).cast(t))

    # 2) Fecha base y particiones
    if date_col and date_col in df.columns:
        if not isinstance(df.schema[date_col].dataType, TimestampType):
            df = df.withColumn(date_col, to_timestamp(col(date_col), date_fmt))
        if extract_date_parts:
            df = (
                df
                .withColumn("year",  year(col(date_col)).cast("string"))
                .withColumn("month", month(col(date_col)).cast("string"))
                .withColumn("day",   dayofmonth(col(date_col)).cast("string"))
            )

    # 3) Fill de nulos (incl. timestamps)
    if fillna_map:
        ts_cols = [c for c, _ in fillna_map.items()
                   if c in df.columns and isinstance(df.schema[c].dataType, TimestampType)]
        # no-timestamps
        plain_map = {c: v for c, v in fillna_map.items() if c in df.columns and c not in ts_cols}
        if plain_map:
            df = df.fillna(plain_map)
        # timestamps
        for c in ts_cols:
            v = fillna_map[c]
            if isinstance(v, str):
                df = df.withColumn(c, when(col(c).isNull(), to_timestamp(lit(v), date_fmt)).otherwise(col(c)))
            else:
                df = df.withColumn(c, when(col(c).isNull(), lit(v).cast("timestamp")).otherwise(col(c)))

    # 4) No negativos
    if non_negative_cols:
        for c in non_negative_cols:
            if c in df.columns:
                zero = _bronze_zero_for_type(df.schema[c].dataType)
                df = df.withColumn(c, when(col(c) < 0, zero).otherwise(col(c)))

    # 5) Drop columns
    if drop_cols:
        keep = [c for c in df.columns if c not in set(drop_cols)]
        df = df.select(*keep)

    # 6) Deduplicación
    if compute_metrics:
        before_dedup = df.count()
    df = df.dropDuplicates(dedup_subset) if dedup_subset else df.dropDuplicates()
    if compute_metrics:
        after_dedup = df.count()
        dq["rows_after_fill_cast"] = before_dedup
        dq["duplicates_removed"] = before_dedup - after_dedup
        dq["rows_out"] = after_dedup

    # --- Salida controlada ---
    if output == "df":
        return df
    # default: "both"
    return df, dq


In [0]:
df_Producto, dq = limpiar_bronze(
    producto,
    casts={"IDPRODUCTO":"int","CODMNDA":"int","CODSUBCAT":"int","MTOPRECUNIT":"double","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"IDPRODUCTO":0,"CODMNDA":0,"CODSUBCAT":0,"MTOPRECUNIT":0.0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["IDPRODUCTO","MTOPRECUNIT"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)


In [0]:
df_Categoria, dq = limpiar_bronze(
    categoria,
    casts={"CODCAT":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODCAT":0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODCAT"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Subcategoria, dq = limpiar_bronze(
    subcategoria,
    casts={"CODSUBCAT":"int","CODCAT":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODSUBCAT":0,"CODCAT":0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODSUBCAT","CODCAT"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Segmento, dq = limpiar_bronze(
    segmento,
    casts={"CODSGMNTO":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODSGMNTO":0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODSGMNTO"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Mercado, dq = limpiar_bronze(
    mercado,
    casts={"CODMRCADO":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODMRCADO":0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODMRCADO"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Ubigeo, dq = limpiar_bronze(
    ubigeo,
    casts={"CODUBIGEO":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODUBIGEO":0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODUBIGEO"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Sectoreconomico, dq = limpiar_bronze(
    SECTORECONOMICO,
    casts={"CODSECTECON":"int","CODSGMNTO":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODSECTECON":0, "CODSGMNTO":0,"FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODSECTECON","CODSGMNTO"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Vendedor, dq = limpiar_bronze(
    VENDEDOR,
    casts={"CODVEND":"int","CODMNDA":"int", "MTOSUELDOBASE":"double", "PCTCOMIS":"double", "FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODVEND":0, "CODMNDA":0, "MTOSUELDOBASE":0, "PCTCOMIS":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODVEND","CODUBIGEO", "MTOSUELDOBASE", "PCTCOMIS"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Moneda, dq = limpiar_bronze(
    MONEDA,
    casts={"CODMNDA":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODMNDA":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODMNDA"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Prioridadpedido, dq = limpiar_bronze(
    PRIORIDADPEDIDO,
    casts={"CODPRIORPEDI":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODPRIORPEDI":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODPRIORPEDI"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Modalidadventa, dq = limpiar_bronze(
    MODALIDADVENTA,
    casts={"CODMODVALVTA":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODMODVALVTA":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODMODVALVTA"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_ModalidadEnvio, dq = limpiar_bronze(
    MODALIDADENVIO,
    casts={"CODMODALENV":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODMODALENV":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODMODALENV"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Mediopago, dq = limpiar_bronze(
    mediopago,
    casts={"CODMEDIOPAGO":"int","FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"CODMEDIOPAGO":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["CODMEDIOPAGO"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
df_Pedido, dq = limpiar_bronze(
    PEDIDO,
    casts={"PEDIDO":"int", "CTDPEDID":"int", "CODMODVALVTA":"int", "CODMEDIOPAGO":"int", "CODVEND":"int", "CODMNDA":"int", "IDCLI":"int", "IDPRODUCTO":"int", 
           "CODMODALENV":"int", "CODPRIORPEDI":"int", "MTOVALUNIT":"double", "MTOSUBT":"double", "MTODSCTO":"double", "MTOBENEF":"double", "MTOVALVTA":"double",
           "MTOIGV":"double", "MTOSUBTPROD":"double", "MTOCSTOENV":"double", "MTOTOTPROD":"double", "FECPEDID":"timestamp", "FECENV":"timestamp",
           "FECCARGA":"timestamp"},
    date_col="FECCARGA",
    fillna_map={"PEDIDO":0, "CTDPEDID":0, "CODMODVALVTA":0, "CODMEDIOPAGO":0, "FECCARGA":"1970-01-01 00:00:00"},
    non_negative_cols=["PEDIDO"],
    compute_metrics=True,        # <- calcula rows_in, duplicates_removed, etc.
    output="both"                # <- retorna (df, dq)
)

In [0]:
%sql
USE CATALOG desarrollo;

SHOW DATABASES;

In [0]:
from typing import List, Optional
from delta.tables import DeltaTable

def crear_tabla_delta_merge_managed(
    nombre_df: str,
    nombre_tabla: str,
    llave_origen: List[str],
    llave_destino: List[str],
    db_name: str = "default",
    catalog_name: str = "desarrollo",
    partition_cols: Optional[List[str]] = None,
    auto_merge_schema: bool = True
) -> None:
    """
    Crea si no existe una tabla Delta GESTIONADA en la base (que ya debe tener LOCATION en tu mount)
    y realiza MERGE. No usa LOCATION explícito.
    """

    # Validaciones
    df = globals()[nombre_df]

    if len(llave_origen) != len(llave_destino):
        print("❌ Error: La cantidad de columnas en 'llave_origen' y 'llave_destino' no coinciden.")
        return

    if partition_cols:
        faltantes = [c for c in partition_cols if c not in df.columns]
        if faltantes:
            print(f"❌ Error: Columnas de partición no existen en el DataFrame: {faltantes}")
            return

    if auto_merge_schema:
        spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Armar nombre completo
    full_name = f"{catalog_name}.{db_name}.{nombre_tabla}"

    # ✅ FIX: usar el overload moderno (una sola cadena)
    exists = spark.catalog.tableExists(full_name)

    if not exists:
        # Crear como TABLA GESTIONADA en el LOCATION de la DB (sin LOCATION explícito)
        writer = df.write.format("delta").mode("overwrite")
        if partition_cols:
            # ✅ FIX: varargs
            writer = writer.partitionBy(*partition_cols)
        writer.saveAsTable(full_name)
        print(f"✅ Tabla gestionada creada: {full_name} (bajo LOCATION de la base '{db_name}')")
        return

    # Si existe, MERGE
    try:
        delta_tbl = DeltaTable.forName(spark, full_name)
    except Exception as e:
        raise RuntimeError(f"❌ La tabla {full_name} no es Delta o no es accesible como Delta: {e}")

    merge_condition = " AND ".join(
        [f"tgt.`{llave_destino[i]}` = src.`{llave_origen[i]}`" for i in range(len(llave_origen))]
    )
    set_expr  = {c: f"src.`{c}`" for c in df.columns}
    vals_expr = {c: f"src.`{c}`" for c in df.columns}

    print(f"🔄 Ejecutando MERGE INTO {full_name} ...")
    (delta_tbl.alias("tgt")
             .merge(df.alias("src"), merge_condition)
             .whenMatchedUpdate(set=set_expr)
             .whenNotMatchedInsert(values=vals_expr)
             .execute())
    print(f"✅ MERGE completado para {full_name}")

In [0]:
# Ejecutar la función para crear la tabla y hacer MERGE usando diferentes llaves
crear_tabla_delta_merge_managed(
    nombre_df="df_Subcategoria",
    nombre_tabla="t_Subcategoria",
    llave_origen=["CODSUBCAT"],
    llave_destino=["CODSUBCAT"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)


In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Categoria",
    nombre_tabla="t_Categoria",
    llave_origen=["CODCAT"],
    llave_destino=["CODCAT"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)


In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Producto",
    nombre_tabla="t_Producto",
    llave_origen=["IDPRODUCTO"],
    llave_destino=["IDPRODUCTO"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Moneda",
    nombre_tabla="t_Moneda",
    llave_origen=["CODMNDA"],
    llave_destino=["CODMNDA"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Ubigeo",
    nombre_tabla="t_Ubigeo",
    llave_origen=["CODUBIGEO"],
    llave_destino=["CODUBIGEO"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Segmento",
    nombre_tabla="t_Segmento",
    llave_origen=["CODSGMNTO"],
    llave_destino=["CODSGMNTO"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Sectoreconomico",
    nombre_tabla="t_SectorEconomico",
    llave_origen=["CODSECTECON"],
    llave_destino=["CODSECTECON"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Mercado",
    nombre_tabla="t_Mercado",
    llave_origen=["CODMRCADO"],
    llave_destino=["CODMRCADO"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
# crear_tabla_delta_merge_managed(
#     nombre_df="df_Cliente",
#     nombre_tabla="t_Cliente",
#     llave_origen=["IDCLI"],
#     llave_destino=["IDCLI"],
#     db_name="bronze_ventas",
#     partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
# )

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Vendedor",
    nombre_tabla="t_Vendedor",
    llave_origen=["CODVEND"],
    llave_destino=["CODVEND"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Prioridadpedido",
    nombre_tabla="t_Prioridadpedido",
    llave_origen=["CODPRIORPEDI"],
    llave_destino=["CODPRIORPEDI"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_ModalidadEnvio",
    nombre_tabla="t_ModalidadEnvio",
    llave_origen=["CODMODALENV"],
    llave_destino=["CODMODALENV"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Modalidadventa",
    nombre_tabla="t_Modalidadventa",
    llave_origen=["CODMODVALVTA"],
    llave_destino=["CODMODVALVTA"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Mediopago",
    nombre_tabla="t_mediopago",
    llave_origen=["CODMEDIOPAGO"],
    llave_destino=["CODMEDIOPAGO"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)

In [0]:
crear_tabla_delta_merge_managed(
    nombre_df="df_Pedido",
    nombre_tabla="t_Pedido",
    llave_origen=["IDPEDIDO"],
    llave_destino=["IDPEDIDO"],
    db_name="bronze_ventas",
    partition_cols=["FECCARGA"]  # opcional; si no quieres partición, quítalo
)